<h1><center>Module 3: Geoprocessing in ArcGIS via Python</center></h1>

With fundamental knowledge of Python and the ability to navigate workspaces, we can begin to work with various geoprocessing techniques made available by ArcPy. When working on a machine with the correct Python environment variables set, a solid understanding of the various tools ArcPy offers can lead to a fully automated mapping process. This module will introduce various common geoprocessing tools native to ArcPy through the Python Window, as well as address common data manipulation techniques and issues you may encounter.    

### 1.1 Clip Analysis
Arguably one of the most commonly employed GIS geoprocessing tools is the [Clip tool](http://desktop.arcgis.com/en/arcmap/10.3/tools/analysis-toolbox/clip.htm), whose name fully outlines its purpose: to clip particular data from one set to another. This can be used to aggregate attribute data by location. For example, the "Directional_Lines" shapefile from the previous module contains a lot of attribute data related to the well itself, but not so much in terms of actual spatial location information. For example, say you were tasked with finding all of the planned wells in Adams county, Colorado when the "Wells" shapefile does not contain county information. In this case, you may choose to employ a clip on the "Directional_Lines" shapefile using the Adams County polygon from the "Colorado_County_Boundaries" shapefile. See the below walkthrough for the process:
<pre><code> 
 >>> import arcpy
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> arcpy.env.workspace = mxd.filePath[:-12]
 >>> arcpy.MakeFeatureLayer_management("Colorado_County_Boundaries", "Adams_County", "County = 'ADAMS'")
 >>> arcpy.MakeFeatureLayer_management("Directional_Lines", "Planned_Directionals", "Dir_status = 'Planned'")
 >>> arcpy.Clip_analysis("Planned_Directionals", "Adams_County", arcpy.env.workspace+"\\Adams_Planned_Directionals.shp")
 >>> del mxd
</code></pre>

The first three lines of the above code should look familiar from the past modules - import the arcpy module, establish your target map document, and establish your workspace. The reason for defining the workspace in addition to the map document is for readability of the code, and is required for defining your output path for the clipped shapefile. You should also recognize the creation of the "Adams_County" temporary layer via the MakeFeatureLayer_management utility, with its associated SQL query for limiting the output to only Adams County; the same parallel should be drawn for the "Planned_Directionals" temporary layer (as you did in the previous project). The final line simply reads in the "Planned_Directionals" layer as the layer to be clipped, with the temporary "Adams_county" layer as the template used to clip and the relative path of the current workspace with the name of the output file. In this case it is important to establish the format of the output file, and in keeping with the theme I choose to output as a shapefile. Following this procedure, you should be presented with the following: 
<img src="figs/adamsPlanned.JPG" width="1000">

Note that I did not save my map document following this procedure. This is because the Clip analysis creates a shapefile which is **not** a temporary file, and for my current purposes I am not interested in preserving the "Adams_County" or "Planned_Directionals" temporary layers. 

This Clip process is especially useful when trimming down massive files when you are only interested in particular locations, such as counties, states, and **PLSS** denominations. PLSS stands for Public Land Survey System, and is a means of dividing up large areas of land (such as states) into more manageable denominations. This is typically accomplished through breaking up the entire area into a series of **Townships** and **Ranges** which represent the location of the area with respect to North/South and East/West of the principal meridian, respectively. Each Township-Range has 36 **Sections** associated with it which are 1 square mile, and count beginning in the Northeast corner and snake west to east, as shown in the figure below:

<p>
    <img src="figs/plssinfo.JPG" width="500"/>
    <br>
    <center><em>Sourced from: https://nationalmap.gov/small_scale/a_plss.html</em></center>
</p>

You can learn more about the PLSS from the [USGS](https://nationalmap.gov/small_scale/a_plss.html) which will be incredibly useful if you are interested in going into the oil and gas field. 

### 1.2 Buffer Analysis
Buffering is another tool which sees a lot of use in ArcMap due to its versatility and ability. As you might imagine from the name, the Buffer tool creates a buffer around particular features with a defined size. This can be useful in a multitude of instances, such as modeling contamination spreading, light dispersion, noise radii; anything which involves a zone of interest around the target. In the case of oil and gas, this can be especially useful for determining pooling of horizontal wellbores. In short, pooling refers to the combination of multiple tracts of land into a single unit for the ease of drilling wells, and the avoidance of surface disturbance. This is the case in terms of the mineral rights, however the geologic basis is this: when a well is drilled, it does not simply produce from the wellbore alone. Rather, hydrocarbons from the target formation are extracted in the same way that a water reservoir is drained. Therefore, when a horizontal well is drilled it is helpful to model the potential pooling area following conventional pooling distances - a perfect utility for the Buffer tool. This can be implemented in the following way:

<pre><code> 
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> arcpy.Buffer_analysis("Adams_Planned_Directionals", arcpy.env.workspace + "\\Buffered_Adams_Planned_Directionals.shp", "200 Feet", method="GEODESIC")
 >>> del mxd
</code></pre>

The above code is pretty straightforward, reading in the input, output (with relative path), buffer distance, and the optional method of calculating the buffer. In this case I chose the "GEODESIC" method because it will preserve the shape of the input shapes, regardless of the coordinate system applied. This is particularly useful as the horizontal well bores typically have angluar features from the surface location to allow for a wider spread of well tracks. There are multiple optional parameters for the buffer tool which can be particularly useful for specific cases, which can be viewed from the [following documentation](http://desktop.arcgis.com/en/arcmap/10.3/tools/analysis-toolbox/buffer.htm). Running the above code should result in the following buffered well tracks:
<img src="figs/buffered.JPG" width="1000px">

The buffer tool is best used in conjunction with additional tools, as it can act as an intermediate step for further spatial analysis as will be described later on in this module.

### 1.3 Intersect 
This geoprocessing tool can be useful when the attributes of the data that you are examining do not contain the same information, but you are interested in where they spatially intersect. Specifically, this tool uses the geometries of the input features to determine where they overlap, and compiling a new feature class which contains the attribute information for all feature classes chosen to be intersected. An example of where this can be useful is tying horizontal wellbore information to specific PLSS sections rather than simply sticks on a map. See below for the implementation of this:
<pre><code> 
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> arcpy.Intersect_analysis(["Planned_Directionals", "CO_Sections"],"Planned_Directionals_w_Sections", cluster_tolerance = "10 Feet", output_type="Input")
 >>> del mxd
</code></pre>

The intersect tool expects a list of input features to run the analysis on (enclosed in square brackets, as you would in Python), an output file name, and a series of optional parameters which can be found [here](http://desktop.arcgis.com/en/arcmap/10.3/tools/analysis-toolbox/intersect.htm). In the above example I established the "cluster_tolerance" which establishes how close the features have to be to be considered intersecting, along with the "output_type" set to mimic that of the input. This helps to ensure that the geometry stays consistent between operations, and when set to "input", the operation will pick the lowest dimension geometry of the input features as the output. This can be a useful way to correlate across reference frames, where attribute tables contain different information for the same geographic locations.

### 1.4 Repairing Geometries and Projections
As you work through increasingly more complex geoprocessing tools you may be required to repair geometries following operations; potentially even upon receiving data that may have had issues during processing before being made available. Whatever the reason, repairing geometries is a critical skill and can be incorporated through Python. Fortunately, this can be handled in a very straightforward fashion requiring only one line of code! See below:
<pre><code> 
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> arcpy.RepairGeometry_management("Planned_Directionals")
 >>> del mxd
</code></pre>
When debugging ArcPy scripts, it is a good first step when you encounter the dreaded "Error 999999: Something unexpected caused the tool to fail."

Projections are another crucial aspect to GIS that must be kept in mind when consolidating data from multiple sources into a singular map. When data is compiled, it is typically done so with respect to a specific coordinate system (of which there are many), and in order to represent data accurately in the same dataframe it is important that they share the same coordinate system. This is where the ArcMap Project Tool comes in - an interface which allows you to **project** data from one coordinate system to another, allowing you to reference these datasets geospatially. The name is coined from the idea of typical map projections such as cylindrical, of which the Mercator projection (typical map shape for Earth maps) is one. You can read up on the concept of map projections and coordinate systems on the ArcGis site [here](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/what-are-map-projections.htm) for more information. 

In ArcGIS Pro, data is projected onto the same coordinate system on the fly in relation to the first data layer, however basic ArcMap installations do not. Because of this, it is beneficial to have an automated process for projecting your downloaded datasets into the same coordinate system for future processing. The following outlines how you might choose to accomplish that:
<pre><code> 
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> targetCS = arcpy.SpatialReference('NAD 1983 UTM Zone 13N')
 >>> arcpy.Project_management("Adams_County", "Adams_Projected", outCS)
 >>> arcpy.MakeFeatureLayer_management(arcpy.env.workspace + "\\Adams_Projected.shp", "Adams_Projected")
 >>> del mxd
</code></pre>

In order to project data from one coordinate system to another, we need to establish a spatial reference object which occurs on the second line. This can be accomplished in a few ways, but most easily through knowing the name of the target coordinate system. In the above example I employed the commonly used "NAD 1983 UTM Zone 13N" system. Establishing this as a named variable will allow for future batching of projection actions for a series of shapefiles which will speed up the process. One quick way to determine what coordinate system your data is in is to simply right click the layers as they appear in the Table of Contents, then selecting "Properties" and viewing the "Source" tab which contains the relevent coordinate system information.

### 1.5 Search and Update Cursors
ArcPy has the ability to comb through attribute tables and both search and update values within those tables. This is controlled through **cursors**, of which there are three types: 
1. Search cursors
2. Update cursors
3. Insert cursors

This series of modules will focus mostly on Search and Update cursors, as those allow you to access rows in attribute tables and modify/delete values in those rows. Cursors are apart of the *arcppy.da* module and need to be initiated to begin facilitating data management operations. 

When a Search Cursor is initiated, it is in essence a list of the rows of a given attribute table for a specific Feature Class. For example, running the following code should output each County name in Colorado:
<pre><code>
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> cursor = arcpy.da.SearchCursor("Colorado_County_Boundaries", ["COUNTY"])
 >>> for row in cursor: print("County = " + str(row[0]))
 >>> del row
 >>> del cursor
</code></pre>

When creating a Search Cursor, you need to decide which fields you are interested in searching through. This is the second parameter in the call, following the target Feature Class. Notice in the above code it is closed by deleting both the "cursor" and the "row" - this is done for a similar reason that the mapping document is closed out each time an operation in completed. When executing scripts in ArcPy a lock is placed on the Feature Classes in question which needs to be removed for additional operations. When looping over cursors, they operate differently than lists; namely in that the loop actually created a "row" object which housed all of the attribute data per row for the attribute table that was looped through, including the specified field information from the cursor. This object needs to be deleted upon the completion of the loop to allow further editing operations to occur. The deletion step is really only crucial when using the Insert and Update cursors, however falling into the habit of deleting references is good practice.

Update Cursors can be initiated in an identical fashion as Search Cursors, however they have the power to actually modify data in the target Feature Class's attribute table. For example, lets say we wanted to rewrite the County Names in the "CO_County_Boundaries" shapefile to be proper titles (capital first letter, not all capitals). We could employ the following Update Cursor:
<pre><code>
 mxd = arcpy.mapping.MapDocument("CURRENT")
 cursor = arcpy.da.UpdateCursor
 for row in cursor: 
     row[0] = row[0].title()
     cursor.updateRow(row[0])
 del row
 del cursor
 del mxd
</code></pre>

Notice how the above code does not look like the previous codeblocks - this is because multiline 'For' loops cannot be input on the command line interface present in the Python Window of ArcMap. This is going to require writing a standalone script which we can run using ArcMaps version of Python containing ArcPy.

While separate from cursors, it is beneficial to be able to programmatically add attributes to your data in the form of new columns. This is another one liner, however there are many important parameters to pass in to effectively manage your new field:
<pre><code>
 >>> arcpy.AddField_management(inFeature, fieldName, fieldType)
</code></pre>

Above are the only required parameters: the Feature Class to add the attribute to, the name of the attribute, and the field type (Integer, String, Long, etc). Make sure to check out the documentation page located [here](http://desktop.arcgis.com/en/arcmap/10.3/tools/data-management-toolbox/add-field.htm) for the full list of optional parameters, however below is a list of what I have found to be the most important to include:

| Optional Parameter |                                                                                      Description                                                                                      |
|:------------------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
|     field_alias    |                                              A more descriptive name for the field you are creating. This explains beyond the field name.                                             |
|  field_is_nullable | Defines whether or not this field can contain 'NULL' values. Important to note that 'NULL' values are not zeros or empty fields, and can result in calculation errors down the road.  |
|   field_precision  |                               This defines the number of digits that the field can hold per  single entry. This can help to clean up calculated fields.                               |


### 1.6 Writing and Running Standalone Scripts for ArcPy
Writing standalone scripts for ArcMap follows the same lines as writing any other Python scripts, only requiring that a specific python.exe is established when executing the script. This location can vary depending on install, but it is typically somewhere along the following lines: "C:\Python27\ArcGIS10.X\python.exe". At the time of authoring, the CSM lab computers has this executable located at the following path: "C:\sw\arcPython\ArcGIS10.7\python.exe". Therefore, in order to execute a standalone script which utilizes ArcPy you would run the following in a Windows command prompt window:
<pre><code>
 > C:\sw\arcPython\ArcGIS10.7\python.exe Script.py
</code></pre>

If you have been working on a computer where the virtual environment was setup successfully and ArcPy is able to be imported via your active Jupyter Notebook, then you may write and execute the following script from directly within this notebook. Otherwise, you may recreate the following lines in Sublime, save as a .py, and execute the script as outlined previously:

In [ ]:
import arcpy
mapPath = #Replace this with the absolute path of your MXD
mxd = arcpy.mapping.MapDocument(mapPath)
arcpy.env.workspace = mxd.filePath[:-12]

cursor = arcpy.da.UpdateCursor("Colorado_County_Boundaries", ["COUNTY"])
for row in cursor: 
    row[0] = row[0].title()
    cursor.updateRow(row)
del row
del cursor
del mxd

**Note:** the above script, whether in a Jupyter Notebook or a standalone Python script, will result in an error if your target map document is currently open on your computer. This is a prime example of the lock that ArcMap places on files that are currently in use, and prevents the editing of files that may already be in an edit session elsewhere. Now, with these tools at your disposal you should be prepared to complete the following project:

### 1.7 Module 3 Project:
Similar to the project in Module 1, You have been tasked with writing a script to generate API10 for horizontal wells. However, you are only asked to do so for wells that are within the following Township/Ranges: 
 * 10N 58W
 * 10N 59W
 * 10N 60W

Using the skills you have learned in the Modules thus far, output a CSV document which contains all horizontal wells (regardless of permit status) from the "Directional Wells" shapefile that lie within the given TR, along with their calculated API10 values using cursors. 

In [1]:
import arcpy
import os
mxd = arcpy.mapping.MapDocument(os.getcwd() + "\\CO_Wells\\CO_Wells.mxd")

In [14]:
# Establish the working environment
default_dir = mxd.filePath[:-12] 
arcpy.env.workspace = default_dir
arcpy.env.overwriteOutput = True

# Create the temporary TR layer
arcpy.MakeFeatureLayer_management("CO_Townships.shp", "Target_TR", "TWNSHPLAB = '10N 58W' OR TWNSHPLAB = '10N 59W' OR TWNSHPLAB = '10N 60W'")

# Clip the Total Directional Lines shp to only those in the TR
arcpy.Clip_analysis("Directional_Lines.shp", "Target_TR", arcpy.env.workspace+"\\TR_Directionals.shp")

# Add the field to the attribute table
newFieldName = "API10"
arcpy.AddField_management(default_dir + "\\TR_Directionals.shp", newFieldName, "TEXT", "", "", "", "Custom API10", "NON_NULLABLE") 


# Create Search and Update cursors to comb through the data and create APIS
# Make empty list for storing new APIS
APIs = []

# Establish Feature class name and field name
fc = arcpy.env.workspace + "\\TR_Directionals.shp"
fieldName = 'API'

# Index to keep track of new API location in relation to attribute table  
index = 0

# Use Search cursor to locate APIS and determine how each needs to be modified
with arcpy.da.SearchCursor(fc, fieldName) as cursor:
    for row in cursor:
        if(len(row[0]) == 10):
            APIs.append("05" + str(row[0][:-2]))
        else:
            APIs.append("05" + str(row[0]))
    del row
    del cursor
    
# Use update cursor to locate each row and assign the corresponding API10
with arcpy.da.UpdateCursor(fc, newFieldName) as cursor:
    for row in cursor:
        row[0] = APIs[index]
        index += 1
        cursor.updateRow(row)
    del row
    del cursor
    